#  Import necessary libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Set plot style for better aesthetics

In [3]:
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (15, 7)

# Load the Data using NumPy

In [4]:
try:
    datos_precios = np.loadtxt(
        "manzanas.csv",      
        delimiter=",",       
        skiprows=1,          
        usecols=np.arange(1, 85) 
    )
    print("Data loaded successfully!")
except IOError:
    print("Error: 'manzanas.csv' not found. Please make sure the file is in the same directory.")
    # In case of error, create dummy data to allow the rest of the script to run
    datos_precios = np.random.rand(5, 84) * 100 + 50

Data loaded successfully!


# Initial Data Inspection

In [5]:
# Using the .ndim attribute to see the number of dimensions of our array.
print(f"Number of dimensions: {datos_precios.ndim}")

# Using the .shape attribute to see the form of our array (rows, columns).
print(f"Shape of the array: {datos_precios.shape}")
print("\nThis shows we have data for 5 cities (rows) over 84 months (columns).")

Number of dimensions: 2
Shape of the array: (5, 84)

This shows we have data for 5 cities (rows) over 84 months (columns).


# Transpose the Data for Time-Series Analysis

In [6]:
datos_transpuestos = datos_precios.T
print(f"Shape of transposed array: {datos_transpuestos.shape}")
print("\nNow we have 84 months (rows) and 5 cities (columns).")

Shape of transposed array: (84, 5)

Now we have 84 months (rows) and 5 cities (columns).
